In [ ]:
# 연습문제 : 지마켓 분야별 베스트 상품 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import random
import os

import urllib.request
import urllib


#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.

print("=" *80)
print("연습문제 : 지마켓 Best Seller 상품 정보 추출하기 ")
print("=" *80)

query_txt = 'G마켓'
query_url='http://corners.gmarket.co.kr/Bestsellers'

cnt = int(input('1.크롤링 할 건수는 몇건입니까?(최대 200건): '))
page_cnt = math.ceil(cnt / 20)

f_dir = input('2.파일을 저장할 폴더명만 쓰세요(예:c:\\py_temp\\):')
if f_dir =='' :
    f_dir = "c:\\py_temp\\"
    
print("\n")

if cnt > 30 :
      print("  요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~")
else :
      print("  요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")
      
    # 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

img_dir = f_dir+s+'-'+query_txt+"\\images"

os.makedirs(img_dir)

os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

s_time = time.time( )

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

driver.get(query_url)
time.sleep(1)

# def scroll_down(driver):
#     driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
#     time.sleep(1)

def scroll_down(driver):      
    driver.execute_script("window.scrollBy(0,2500);")
    time.sleep(4)
    
for cc in range(1,4) :
    scroll_down(driver)
        
# 내용 수집하기

ranking2=[]
title2=[]
o_price2=[]
s_price2=[]
discount2=[]

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
        
count = 1

slist = soup.find('div','best-list').find_all('li')


#이미지 추출하기
img_file_no = 1

for li in slist:
                
    os.chdir(img_dir)

    try :
          photo = li.find('div','thumb').find('img')['src']
    except AttributeError :
        continue
    
    print('%s번째 이미지를 저장하고 있으니 잠시 기다려 주세요~~' %img_file_no)
    photo2 = 'http:' + photo
    urllib.request.urlretrieve(photo2,str(img_file_no)+'.jpg')
    time.sleep(2)

    if img_file_no > cnt :
        break
        
    img_file_no += 1

# 텍스트 추출하기

for li in slist:
            
    f = open(ff_name, 'a',encoding='UTF-8')
    f.write("-----------------------------------------------------"+"\n")
    
    print("\n")
    print("-" *70)
    
    #판매순위
    sid = '#no' + str(count)
    try :
        ranking = li.select(sid)[0].get_text()
    except AttributeError :
        ranking = ''
    
    print('1.판매순위:',ranking.replace("\n",""))
    f.write('1.판매순위:'+ ranking + "\n")
    ranking2.append(ranking)

    #제품이름
    try :
        title = li.select('a.itemname')[0].get_text()
    except AttributeError :
        title = ''

    print("2.제품이름:", title.replace("\n",""))
    f.write('2.제품이름:'+ title + "\n")
    title2.append(title.replace("\n",""))

    # 원래 가격
   
    o_price = li.find('div', class_='item_price').find('div','o-price').get_text()
    if o_price == '':
        o_price = li.find('div', class_='item_price').find('div','s-price').find('strong').get_text()
        
    print("3.원래가격:", o_price.replace("\n",""))
    f.write('3.원래가격:'+ o_price + "\n")
    o_price2.append(o_price.replace("\n",""))
    
    # 판매 가격
    s_price = li.find('div', class_='item_price').find('div','s-price').find('strong').get_text()
    print("4.판매가격:", s_price.replace("\n",""))
    f.write('4.판매가격:'+ s_price + "\n")
    s_price2.append(s_price.replace("\n",""))

    # 할인율
    try :
        discount = li.find('div', class_='item_price').find('div','s-price').find('em').get_text()
    except :
        discount = '0%'

    print("5.할인율:", discount.replace("\n",""))
    f.write('5.할인율:'+ discount + "\n")
    discount2.append(discount.replace("\n",""))     

    if count == cnt :
        break

    count += 1

    time.sleep(0.5)


#step 9. 엑셀 형태로 저장하기
              
g_best_seller = pd.DataFrame()

g_best_seller['판매순위']=ranking2
g_best_seller['제품소개']=pd.Series(title2)
g_best_seller['원래가격']=pd.Series(o_price2)
g_best_seller['판매가격']=pd.Series(s_price2)
g_best_seller['할인율']=pd.Series(discount2)

# csv 형태로 저장하기
g_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
g_best_seller.to_excel(fx_name ,index=True , engine='openpyxl')

# 요약정보 출력하기  

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 요약 정보 출력
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 건수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)
     
#이미지 삽입하기
import win32com.client as win32   
import win32api  
                
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
    col_name='C'+str(a)
    col_name2.append(col_name)

for b in range(1,cnt+1) :
    file_name=img_dir+'\\'+str(b)+'.jpg'
    file_name2.append(file_name)
      
for i in range(0,cnt) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
    excel.Visible=True
    excel.ActiveWorkbook.Save()

#driver.close()


연습문제 : 지마켓 Best Seller 상품 정보 추출하기 
1.크롤링 할 건수는 몇건입니까?(최대 200건): 10
2.파일을 저장할 폴더명만 쓰세요(예:c:\py_temp\):


  요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~
1번째 이미지를 저장하고 있으니 잠시 기다려 주세요~~
2번째 이미지를 저장하고 있으니 잠시 기다려 주세요~~
3번째 이미지를 저장하고 있으니 잠시 기다려 주세요~~
4번째 이미지를 저장하고 있으니 잠시 기다려 주세요~~
5번째 이미지를 저장하고 있으니 잠시 기다려 주세요~~
6번째 이미지를 저장하고 있으니 잠시 기다려 주세요~~
7번째 이미지를 저장하고 있으니 잠시 기다려 주세요~~
8번째 이미지를 저장하고 있으니 잠시 기다려 주세요~~
9번째 이미지를 저장하고 있으니 잠시 기다려 주세요~~
